In [1]:
%load_ext autoreload
%autoreload all
import os
os.chdir("..")
import pandas as pd
import anndata
import time
from tqdm.notebook import tqdm
from Utils.Settings import family_name,class_to_division, class_to_broad_division,neuron_cluster_groups_order, manifest, download_base, output_folder_calculations, output_folder
from Utils.Utils import broad_division_color_map, cluster_groups_cmap, percentage_above_threshold_MER, percentage_above_threshold
from pathlib import Path
import numpy as np

# Load data

In [2]:
metadata = manifest['file_listing']['WMB-10X']['metadata']

In [3]:
rpath = metadata['cell_metadata_with_cluster_annotation']['files']['csv']['relative_path']
file = os.path.join( download_base, rpath )
cell = pd.read_csv(file, keep_default_na=False)
cell.set_index('cell_label',inplace=True)

In [4]:
matrices = cell.groupby(['dataset_label','feature_matrix_label'])[['library_label']].count()
matrices.columns  = ['cell_count']


In [5]:
rpath = metadata['example_genes_all_cells_expression']['files']['csv']['relative_path']
file = os.path.join( download_base, rpath.replace('example_genes_all_cells_expression.csv', f'{family_name}_genes_all_cells_expression.csv'))
exp = pd.read_csv(file)
exp.set_index('cell_label',inplace=True)
exp = exp.sort_index(axis=1)


# Process

In [6]:
cell["division"] = cell['class'].map(class_to_division)
cell["broad_division"] = cell['class'].map(class_to_broad_division)

In [7]:
metadata = manifest['file_listing']['WMB-neighborhoods']['metadata']
rpath = metadata['cluster_group_membership']['files']['csv']['relative_path']
file = os.path.join( download_base, rpath)
group_membership = pd.read_csv(file) # cluster can belong to two different groups


In [8]:
cell_with_membership = cell.reset_index().merge(group_membership[["cluster_group_label", "cluster_alias", "cluster_group_name"]], on='cluster_alias').set_index("cell_label")


In [9]:
cell_with_membership["cluster_group_color"] = cell_with_membership["cluster_group_name"].map(cluster_groups_cmap)
cell['broad_division_color'] = cell['broad_division'].map(broad_division_color_map)

In [10]:
selected_genes = exp.columns.sort_values()

joined = cell.join(exp)
joined_boolean =  cell.join( exp.astype("bool") )
subsampled = joined.loc[::30]

In [11]:
joined_with_membership = cell_with_membership.join(exp)

joined_boolean_with_membership =  cell_with_membership.join( exp.astype("bool") )
subsampled_with_membership = joined_with_membership.loc[::30]

The prerequisite for running this notebook is that the data have been downloaded to local directory maintaining the organization from the manifest.json. **Change the download_base variable to where you have downloaded the data in your system.**

In [12]:
datasets = ['Zhuang-ABCA-1','Zhuang-ABCA-2','Zhuang-ABCA-3','Zhuang-ABCA-4']

metadata = {}
for d in datasets :
    metadata[d] = manifest['file_listing'][d]['metadata']

# Load data 2

In [13]:
cell = {}

for d in datasets :
    
    rpath = metadata[d]['cell_metadata']['files']['csv']['relative_path']
    file = os.path.join( download_base, rpath)
    cell[d] = pd.read_csv(file, dtype={"cell_label":str})
    cell[d].set_index('cell_label',inplace=True)
    
    sdf = cell[d].groupby('brain_section_label')
    
    print(d,":","Number of cells = ", len(cell[d]), ", ", "Number of sections =", len(sdf))

Zhuang-ABCA-1 : Number of cells =  2846908 ,  Number of sections = 147
Zhuang-ABCA-2 : Number of cells =  1227408 ,  Number of sections = 66
Zhuang-ABCA-3 : Number of cells =  1585843 ,  Number of sections = 23
Zhuang-ABCA-4 : Number of cells =  162578 ,  Number of sections = 3


### Cluster annotation

Read in the pivot table from the "cluster annotation tutorial" to associate each cell with terms at each cell type classification level and the corresponding color.

In [14]:
taxonomy_metadata = manifest['file_listing']['WMB-taxonomy']['metadata']

rpath = taxonomy_metadata['cluster_to_cluster_annotation_membership_pivoted']['files']['csv']['relative_path']
file = os.path.join( download_base, rpath)
cluster_details = pd.read_csv(file,keep_default_na=False)
cluster_details.set_index('cluster_alias', inplace=True)

rpath = taxonomy_metadata['cluster_to_cluster_annotation_membership_color']['files']['csv']['relative_path']
file = os.path.join( download_base, rpath)
cluster_colors = pd.read_csv(file)
cluster_colors.set_index('cluster_alias', inplace=True)

In [15]:
cell_extended = {}

for d in datasets :
    cell_extended[d] = cell[d].join(cluster_details,on='cluster_alias')
    cell_extended[d] = cell_extended[d].join(cluster_colors,on='cluster_alias')

For convenience, we can cache this view for later reuse.

### Gene panel

All 4 datasets shares the same 1122 gene panel selected to enable faciliate the mapping to transcriptomically defined cell types taxonomies. Each gene is uniquely identifier by an Ensembl ID. It is best practice to gene identifier to for tracking and data interchange as gene symbols are not unique and can change over time.

Each row of the gene dataframe has Ensembl gene identifier, a gene symbol and name.

In [16]:
rpath = metadata[datasets[0]]['gene']['files']['csv']['relative_path']
file = os.path.join( download_base, rpath)
gene = pd.read_csv(file)
gene.set_index('gene_identifier',inplace=True)
print("Number of genes = ", len(gene))


Number of genes =  1122


In [17]:
expression_matrices = {}

for d in datasets :
    expression_matrices[d] = manifest['file_listing'][d]['expression_matrices']

In [18]:

gene_filtered  = pd.read_csv(Path(output_folder_calculations, "selected_genes_MERFISH.csv"))
selected_genes = np.sort(gene_filtered["gene_symbol"].values)


In [19]:
cell_expression = {}
cell_expression_bool = {}

for d in tqdm(datasets):
    
    expression_matrices[d]
    rpath = expression_matrices[d][d]['log2']['files']['h5ad']['relative_path']
    file = os.path.join( download_base, rpath)
    
    adata = anndata.read_h5ad(file,backed='r')
    
    start = time.process_time()
    gdata = adata[:,gene_filtered.index].to_df()
    gdata.columns = gene_filtered.gene_symbol
    cell_expression[d] = cell_extended[d].join( gdata )
    cell_expression_bool[d] = cell_extended[d].join( gdata.astype("bool") )
    
    print(d,"-","time taken: ", time.process_time() - start)
    
    adata.file.close()
    del adata


  0%|          | 0/4 [00:00<?, ?it/s]

Zhuang-ABCA-1 - time taken:  7.083126163999992
Zhuang-ABCA-2 - time taken:  3.8588602059999744
Zhuang-ABCA-3 - time taken:  5.009331188000004
Zhuang-ABCA-4 - time taken:  0.471103276000008


### CCF registration and parcellation annotation

Each brain specimen has been registered to Allen CCFv3 atlas, resulting in an x, y, z coordinates and parcellation_index for each cell. 

In [20]:
ccf_coordinates = {}

for d in tqdm(datasets):
    
    rpath = manifest['file_listing'][d+'-CCF']['metadata']['ccf_coordinates']['files']['csv']['relative_path']
    file = os.path.join( download_base, rpath)
    ccf_coordinates[d] = pd.read_csv(file)
    ccf_coordinates[d].set_index('cell_label',inplace=True)
    ccf_coordinates[d].rename(columns={'x':'x_ccf','y':'y_ccf','z':'z_ccf'},inplace=True)
    
    cell_expression[d] = cell_expression[d].join(ccf_coordinates[d],how='inner')
    cell_expression_bool[d] = cell_expression_bool[d].join(ccf_coordinates[d],how='inner')

  0%|          | 0/4 [00:00<?, ?it/s]

In [21]:
metadata = manifest['file_listing']['Allen-CCF-2020']['metadata']
rpath = metadata['parcellation_to_parcellation_term_membership_acronym']['files']['csv']['relative_path']
file = os.path.join( download_base, rpath)
parcellation_annotation = pd.read_csv(file)
parcellation_annotation.set_index('parcellation_index',inplace=True)
parcellation_annotation.columns = ['parcellation_%s'% x for x in  parcellation_annotation.columns]

rpath = metadata['parcellation_to_parcellation_term_membership_color']['files']['csv']['relative_path']
file = os.path.join( download_base, rpath)
parcellation_color = pd.read_csv(file)
parcellation_color.set_index('parcellation_index',inplace=True)
parcellation_color.columns = ['parcellation_%s'% x for x in  parcellation_color.columns]

In [22]:
for d in datasets :
    cell_expression[d] = cell_expression[d].join(parcellation_annotation,on='parcellation_index')
    cell_expression[d] = cell_expression[d].join(parcellation_color,on='parcellation_index')   
    cell_expression_bool[d] = cell_expression_bool[d].join(parcellation_annotation,on='parcellation_index')
    cell_expression_bool[d] = cell_expression_bool[d].join(parcellation_color,on='parcellation_index')   


In [23]:
import panel as pn
import holoviews as hv
from holoviews import dim, opts

pn.extension()

In [24]:
cell_expression['Zhuang-ABCA-1'].columns

Index(['brain_section_label', 'feature_matrix_label', 'donor_label',
       'donor_genotype', 'donor_sex', 'cluster_alias', 'x', 'y', 'z',
       'subclass_confidence_score', 'cluster_confidence_score',
       'high_quality_transfer', 'neurotransmitter', 'class', 'subclass',
       'supertype', 'cluster', 'neurotransmitter_color', 'class_color',
       'subclass_color', 'supertype_color', 'cluster_color', 'Htr7', 'Htr1f',
       'Htr1d', 'Htr4', 'Htr1b', 'Htr2a', 'Htr1a', 'Htr2c', 'Htr3a', 'x_ccf',
       'y_ccf', 'z_ccf', 'parcellation_index', 'parcellation_organ',
       'parcellation_category', 'parcellation_division',
       'parcellation_structure', 'parcellation_substructure',
       'parcellation_organ_color', 'parcellation_category_color',
       'parcellation_division_color', 'parcellation_structure_color',
       'parcellation_substructure_color'],
      dtype='object')

In [25]:
total_bytes = cell_expression['Zhuang-ABCA-1'][['brain_section_label','parcellation_structure', 'parcellation_substructure','parcellation_division_color','parcellation_division' , "parcellation_structure_color", "Htr2a"]].memory_usage(index=True).sum()
size_in_mb = total_bytes / (1024 ** 2)
size_in_mb

149.70748901367188

In [26]:
# Create a sorted list of unique section names

dataset_changed = pn.widgets.Checkbox(name='Dataset Changed', value=False)


dataset_selector = pn.widgets.Select(name='Dataset Selector', options=datasets, value=datasets[0])

dataset = datasets[0]

section_names = sorted(cell_expression[dataset]['brain_section_label'].unique())

section_name_display = pn.pane.HTML()

# Create a slider for selecting brain sections based on the length of the section_names list
slider = pn.widgets.IntSlider(name='Brain Section Index', start=0, end=len(section_names) - 1, value=0)

gene_selector = pn.widgets.Select(name='Gene Selector', options=selected_genes.tolist(), value=selected_genes[0])


@pn.depends(dataset_selector.param.value, watch=True)
def on_dataset_change(selected_dataset):
    #print(f"Dataset changed to: {selected_dataset}") # Debugging print
    section_names = get_section_names(selected_dataset)
    slider.end = len(section_names) - 1
    #print(f"Slider end set to: {slider.end}") # Debugging print
    slider.param.set_param(value=0)
    #print(f"Slider value set to: {slider.value}") # Debugging print
    dataset_changed.value = True


def get_section_names(dataset):
    return sorted(cell_expression[dataset]['brain_section_label'].unique())

@pn.depends(dataset_selector.param.value, watch=True)
def update_slider_range(selected_dataset):
    section_names = get_section_names(selected_dataset)
    slider.end = len(section_names) - 1


@pn.depends(slider.param.value, dataset_selector.param.value, watch=True)
def update_section_name_display(section_index, dataset=dataset_selector.value):
    section_names = get_section_names(dataset)
    brain_section = section_names[section_index]
    section_name_display.object = f"<b>Brain Section:</b> {brain_section}"


@pn.depends(slider.param.value_throttled, gene_selector.param.value, dataset_selector.param.value)
def plot_expr_across_slices(section_index, gene, dataset):
    _ = cell_expression[dataset]
    
    curve_data = _[_[gene]>0].groupby('brain_section_label')[gene].mean()
    curve_data.rename(f"Expression {gene}", inplace=True)
    curve = hv.Curve(curve_data, label='Gene Expression').opts(xaxis=None, width=1000, height=100, active_tools=[])
    
    grouped_data = cell_expression[dataset].groupby('brain_section_label')[gene].apply(percentage_above_threshold_MER).reset_index()
    grouped_data.rename(columns={gene:"%"}, inplace=True)
    percentage_curve = hv.Curve(grouped_data, 'brain_section_label', "%", label='Percentage').opts(xaxis=None, width=1000, height=100, active_tools=[])
    
    vline = hv.VLine(section_index).opts(line_color='red', line_width=2)
    
    return hv.Layout([(curve * vline), (percentage_curve * vline)]).cols(1)
    
@pn.depends(slider.param.value_throttled, gene_selector.param.value, dataset_selector.param.value)
def plot_section(section_index, gene, dataset):
    section_names = get_section_names(dataset)  
    # Check if the dataset has changed and reset the flag
    
    if dataset_changed.value:
        section_index = 0
        dataset_changed.value = False

    if section_index >= len(section_names):
        section_index = len(section_names) - 1  # Set to the last valid index
        
    brain_section = section_names[section_index]
    section = cell_expression[dataset][cell_expression[dataset]['brain_section_label'] == brain_section]
    section = section[section['parcellation_division'] != "unassigned"]

    plot1 = hv.Points(data=section, kdims=['x', 'y' ], vdims=[gene,'parcellation_structure', 'parcellation_substructure']).opts(width=550,colorbar=True, cmap='rocket_r', size=2, color=gene)
    plot2 = hv.NdOverlay({division: hv.Points(section[section['parcellation_division'] == division], ['x', 'y'], ['parcellation_division_color','parcellation_structure', 'parcellation_substructure']).opts(color='parcellation_division_color', size=2) for division in section[section['parcellation_category'] == "grey"]['parcellation_division'].unique()}, kdims='parcellation_division').opts(legend_position='top_right', width=450,show_legend=True)

    common_opts = opts.Points(height=500,  invert_yaxis=True, xaxis=None, yaxis=None, show_grid=False, active_tools=[], tools=["hover"])

    return (plot1 + plot2).opts(common_opts)

@pn.depends(slider.param.value_throttled, gene_selector.param.value, dataset_selector.param.value)
def plot_bar(section_index, gene, dataset):
    section_names = get_section_names(dataset)
    
    if dataset_changed.value:
        section_index = 0
        dataset_changed.value = False
        
    brain_section = section_names[section_index]
    section = cell_expression[dataset][cell_expression[dataset]['brain_section_label'] == brain_section]
    section = section[section['parcellation_division'] != "unassigned"]
    color_map = section.groupby('parcellation_structure')['parcellation_structure_color'].first().to_dict()

    hv_ds = hv.Dataset(section.groupby(['parcellation_structure'])[gene].apply(percentage_above_threshold_MER).sort_values(ascending=False).head(20).reset_index(), 
                       kdims=['parcellation_structure'], vdims=[gene])
    
    # Create the vertical Bar chart with rotated xticklabels
    bar_plot = hv.Bars(hv_ds).opts(
        color=dim('parcellation_structure').categorize(color_map, 'gray'),
        active_tools=[],
        tools=["hover"],
        frame_width=800,
        frame_height=100,
        invert_axes=False,
        xrotation=90  # Rotate xticklabels by 90 degrees
    )
    return bar_plot
# Update inter to use the combined_plot function

pn.Column(pn.Row(gene_selector, dataset_selector, slider, section_name_display) , pn.Row(plot_expr_across_slices), plot_bar, plot_section)

Column
    [0] Row
        [0] Select(name='Gene Selector', options=['Htr1a', 'Htr1b', ...], value='Htr1a')
        [1] Select(name='Dataset Selector', options=['Zhuang-ABCA-1', ...], value='Zhuang-ABCA-1')
        [2] IntSlider(end=128, name='Brain Section Index')
        [3] HTML(None)
    [1] Row
        [0] ParamFunction(function, _pane=HoloViews, defer_load=False)
    [2] ParamFunction(function, _pane=HoloViews, defer_load=False)
    [3] ParamFunction(function, _pane=HoloViews, defer_load=False)

In [27]:
import ipywidgets as widgets
from ipywidgets import interactive
from ipywidgets import HBox
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.patheffects as path_effects
import matplotlib.colors as mcolors

In [28]:
def plot_class_expression_split_by_categories(subfig,  cls, df_subset, gene):
    axs = subfig.subplots(4,1)
    subfig.subplots_adjust(left=0.05, right=.95)
    
    
    sns.violinplot(data=df_subset, y=f'{gene} prevalence (%)', x='subclass', hue='subclass' ,ax=axs[0], palette= classification_cmap, fill=False)
        
    # Set the title with desired modifications
    title = axs[0].set_title(cls, color=classification_cmap[cls], fontweight='bold', fontsize=20)
    title.set_path_effects([path_effects.withStroke(linewidth=3, foreground='black')])
        
    subfig.set_facecolor(mcolors.to_rgba(classification_cmap[cls], alpha=0.15) )
    
    axs[0].set_ylim(0, 100)  # Set y-axis limit to 1
    axs[0].xaxis.set_ticks_position('top') 
    axs[0].tick_params(axis='x', length=0)
    
    sns.violinplot(data=df_subset, y=f'{gene} prevalence (%)', x='supertype', hue='supertype' ,ax=axs[1], palette= classification_cmap, fill=False)
    axs[1].set_title(cls)
    axs[1].set_ylim(0, 100)  # Set y-axis limit to 1
    
    
    sns.barplot(data=df_subset, y=f'{gene} prevalence (%)', x='cluster',hue='cluster' ,ax=axs[2], palette= classification_cmap)
    axs[2].set_title(cls)
    axs[2].set_ylim(0, 100)  # Set y-axis limit to 1
    
    axs[2].set( ylabel='')
    axs[0].set( ylabel='')
    axs[2].set(xticks=[], xlabel='')
    axs[1].set(xticks=[], xlabel='')
    axs[0].set(xticks=[], xlabel='')
    axs[0].set( xlabel='')
    axs[2].set(xticks=[], xlabel='')
    axs[2].set(xticks=[], xlabel='')
    axs[1].set_title('')
    axs[2].set_title('')
    plt.draw()
    for tick in axs[0].get_xticklabels():
        tick.set_color(classification_cmap[tick.get_text()])
    
    # Identify positions for subclass delimiters in the cluster graph
    subclass_ends = df_subset.groupby('subclass').size().cumsum()
    subclass_positions = subclass_ends - 1
    
    # For the third plot (clusters):
    for pos in subclass_positions[:-1]:
        axs[2].axvline(pos + 0.5, color='gray', linestyle='--')
    
    # For the first plot (subclass), simply draw lines between unique subclasses
    for i in range(1, len(df_subset['subclass'].unique())):
        axs[0].axvline(i - 0.5, color='gray', linestyle='--')
    
    # For the second plot (supertype), the position will be based on unique supertypes per subclass
    supertype_ends = df_subset.groupby('subclass')['supertype'].nunique().cumsum()
    supertype_positions = supertype_ends - 1
    for pos in supertype_positions[:-1]:
        axs[1].axvline(pos + 0.5, color='gray', linestyle='--')
    
    
    # For the first plot (subclass):
    subclasses = df_subset['subclass'].unique()
    start = -.5
    for i, subclass in enumerate(subclasses):
        end = i + 0.5
        color = classification_cmap[subclass]
        axs[0].axvspan(start+.05, end-.05, facecolor=color, alpha=0.15)
        start = end
    
    
    # For the second plot (supertype), determine filling areas based on unique supertypes per subclass
    supertype_ends = df_subset.groupby('subclass')['supertype'].nunique().cumsum()
    supertype_positions = supertype_ends - 1
    
    previous_position = -0.5  # initialize to start from the first entry
    for idx, pos in enumerate(supertype_positions):
        current_position = pos + 0.5
        # Retrieve the subclass name for the current supertype
        subclass_name = df_subset['subclass'].unique()[idx]
        color = classification_cmap[subclass_name]
        
        axs[1].axvspan(previous_position + 0.2, current_position -.2, facecolor=color, alpha=0.15)
    
        previous_position = current_position  # set for the next iteration
    
    ####third plot filling
    # Get a mapping of subclasses to their dominant cluster
    subclass_to_cluster = df_subset.groupby('subclass')['cluster'].apply(lambda x: x.value_counts().idxmax())
    
    # Define previous position to help us fill areas between borders
    prev_pos = 0
    
    # Iterate over the subclass positions and the corresponding subclasses
    for idx, (subclass, cluster) in enumerate(subclass_to_cluster.items()):
        # Get the color for this subclass's dominant cluster from your cmap
        color_value = classification_cmap[cluster]
        
        # End position is either the next border position or the end of the plot
        end_pos = subclass_positions[idx] if idx < len(subclass_positions) else len(df_subset['cluster']) - 1
        
        # Fill the area with the given color
        axs[2].fill_between([prev_pos, end_pos], 0, 100, color=color_value, alpha=0.15)
        
        # Update the previous position to the current position
        prev_pos = end_pos + 1
    
    # Adjust the xlim of each axis
    axs[0].set_xlim(-0.5, len(df_subset['subclass'].unique()) - 0.5)
    axs[1].set_xlim(-0.5, len(df_subset['supertype'].unique()) - 0.5)
    axs[2].set_xlim(-.8, len(df_subset['cluster'].unique()))

    # Now, add legend in the fourth axis:
    patches = [plt.Line2D([0], [0], color=classification_cmap[c], lw=8) for c in df_subset["subclass"].unique()]
    labels = list(df_subset["subclass"].unique())
    
    axs[3].legend(patches, labels, loc='center', ncol=6, handlelength=1.8, handleheight=1)
    axs[3].axis('off')  # Hide axis

    sns.despine(bottom=True)  



In [29]:

total_bytes = joined_with_membership[["cluster_group_name","class","subclass", "supertype", "cluster", "Htr2a"]].memory_usage(index=True).sum()
size_in_mb = total_bytes / (1024 ** 2)
size_in_mb

446.30084228515625

In [30]:
def interactive_plot(group, gene):
    global global_saved_fig, group_g, gene_g
    data = joined_with_membership[joined_with_membership["cluster_group_name"]==group]
    data = data.groupby(["cluster_group_name","class","subclass", "supertype", "cluster"])[gene].apply(percentage_above_threshold).reset_index()
    data = data[data["cluster_group_name"]!="WholeBrain"]
    data["cluster_group_name"] = pd.Categorical(data["cluster_group_name"], categories=neuron_cluster_groups_order, ordered=True)
    data = data.sort_values(by=["cluster_group_name", "class"])

    data.rename(columns={gene: f'{gene} prevalence (%)'}, inplace=True)

    fig = plt.figure(figsize=(25, 5*data['class'].nunique()))
    fig.suptitle(group, fontsize=16)
    gs = gridspec.GridSpec(data['class'].nunique(), 1)

    
    for n, cls in tqdm(enumerate(data['class'].unique()), total=len(data['class'].unique()), desc="Plotting"):
        df_subset = data[data['class'] == cls]
        subfig = fig.add_subfigure(gs[n,:])   
        subfig.subplots_adjust(left=0.05, right=.95, bottom=0.01)
        plot_class_expression_split_by_categories(subfig, cls, df_subset, gene)

    global_saved_fig = fig
    group_g = group
    gene_g = gene
    return fig.suptitle(group, y=1.02, fontweight='bold', fontsize=20);

In [31]:
classification_cmap = joined.drop_duplicates(subset='subclass').set_index('subclass')['subclass_color'].to_dict()
classification_cmap.update(joined.drop_duplicates(subset='supertype').set_index('supertype')['subclass_color'].to_dict())
classification_cmap.update(joined.drop_duplicates(subset='cluster').set_index('cluster')['subclass_color'].to_dict())
classification_cmap.update(joined.drop_duplicates(subset='class').set_index('class')['class_color'].to_dict())


In [32]:
def save_fig():
    if global_saved_fig:
        global_saved_fig.savefig(Path(output_folder, f"{group_g}-{gene_g}.png"))
        print(f"Saved")
    else:
        print("No figure to save")

In [33]:
gene_filtered  = pd.read_csv(Path(output_folder_calculations, "selected_genes_RNAseq.csv"))
selected_genes = np.sort(gene_filtered["gene_symbol"].values)

In [34]:
# Using interact with both gene and group dropdowns
save_button = widgets.Button(description="Save Figure")
save_button.on_click(lambda b: save_fig())


widget1 = widgets.Dropdown(
    options=neuron_cluster_groups_order,
    value=neuron_cluster_groups_order[0],
    description='Group:'
)

widget2 = widgets.Dropdown(
    options=selected_genes,
    value=selected_genes[0],
    description='Gene:'
)

w = interactive(interactive_plot, group=widget1, gene=widget2)

display(HBox([w.children[0], w.children[1], save_button]))
display(w.children[-1])

Output()